In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score
import pandas as pd

import xgboost as xgb

In [2]:
df = pd.read_pickle('data/EventPassArrives.pickle')

In [3]:
df = df.rename(columns={x:"-".join(list(x)) if type(x)==tuple else x for x in df.columns})
df.columns

Index(['nx-CB0', 'nx-CB1', 'nx-CB2', 'nx-CB3', 'nx-CB4', 'nx-DB0', 'nx-DB1',
       'nx-DB2', 'nx-DB3', 'nx-DE0',
       ...
       'yardsToGo', 'down', 'route', 'defTeam', 'offenseFormation',
       'typeDropback', 'defendersInTheBox', 'numberOfPassRushers',
       'absoluteYardlineNumber', 'passResult'],
      dtype='object', length=304)

In [4]:
X = df.query('passResult>=0')[list(df.columns)[:-1]].values
y = df.query('passResult>=0')[list(df.columns)[-1]].values
#y = df.query('bdefense==-1')['ir01'].values

In [5]:
set(y)

{0, 1, 2}

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3)

In [7]:
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

In [8]:
param = {
    'eta': 0.3, 
    'max_depth': 6,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

param1 = {
    'eta': 0.3, 
    'max_depth': 6,  
    'objective': 'binary:logistic',  
    'num_class': 2} 

steps = 40  # The number of training iterations

In [9]:
model = xgb.train(param, D_train, steps)

In [10]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
preds1 = model.predict(D_train)
best_preds = np.asarray([np.argmax(line) for line in preds])
best_preds1 = np.asarray([np.argmax(line) for line in preds1])

print("Preds from Test Data")
print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))
print("Preds from Training Data")
print("Precision = {}".format(precision_score(Y_train, best_preds1, average='macro')))
print("Recall = {}".format(recall_score(Y_train, best_preds1, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_train, best_preds1)))

Preds from Test Data
Precision = 0.46072672303510914
Recall = 0.3922617337169445
Accuracy = 0.7869381673702508
Preds from Training Data
Precision = 0.9589054805165428
Recall = 0.7375624718435841
Accuracy = 0.9025013304949441


In [29]:
preds

array([[0.91054827, 0.07873426, 0.01071744],
       [0.31375512, 0.66936123, 0.01688357],
       [0.9536082 , 0.04174539, 0.00464647],
       ...,
       [0.75522345, 0.21609108, 0.02868549],
       [0.81618536, 0.18004557, 0.00376906],
       [0.91769934, 0.0747347 , 0.00756599]], dtype=float32)